# Smart Home Automation

**Team Number:** 1

**Team Members:**

*   Anirudh Jayan
*   Abhinav Variyath
*   Tara Samiksha
*   Sarvesh Ram Kumar
*   Aravind S Harilal

## Introduction & Problem Statement

**Goal:** Create a Smart Home Automation System to manage electronic devices, optimize electricity consumption, and reduce energy wastage.

**Key Features:**

*   Device management using a **Priority Queue**
*   Automation rule implementation with a **Linked List**
*   Energy-efficient device coordination

## Linked List: Structured Rule Automation System

### What is a Linked List?

A linear data structure where elements (nodes) are linked sequentially. It excels at dynamic insertion and deletion of elements.

### Why Use a Linked List for Automation Rules?

*   **Ordered Rule Execution:**  Automation rules often need to be executed in a specific sequence.
*   **Dynamic Rule Sets:** Easily add, remove, or modify rules without complex restructuring.

### Rule Execution Process

1. Rules are parsed from user input (or external sources) and converted into `Rule` objects.
2. `Rule` objects are added to the `LinkedList`.

In [12]:
import threading 
class Node:  # For creation of a node. Each node contains the data and the address of the next node.
    def __init__(self, val):
        self.val = val
        self.next = None

class LinkedList:  # Linked list implementation.
    def __init__(self):
        self.head = None
        self.size = 0
        self.lock = threading.Lock()

    def add_front(self, val):  # Adding a node in the front of the linked list.
        with self.lock:
            new_node = Node(val)
            new_node.next = self.head
            self.head = new_node
            self.size += 1

    def add_end(self, val):  # Adding a node at the end of the linked list.
        new_node = Node(val)

        with self.lock:
            if not self.head:
                self.head = new_node

            else:
                temp = self.head
                while temp.next:
                    temp = temp.next
                temp.next = new_node
            self.size += 1

    def peek(self):  # Gives us the data in the very first node without popping it.
        with self.lock:
            return self.head.val if self.head else None

    def peek_end(self):  # Gives us the data at the last node without popping it.
        with self.lock:
            if not self.head:
                return None
            temp = self.head
            while temp.next:
                temp = temp.next
            return temp.val

    def get_size(self):  # Gives us the size of the linked list.

        with self.lock:
            return self.size

    def print_list(self):  # Displays the entire list.

        with self.lock:
            temp = self.head
            while temp:
                print(temp.val, end=" -> ")
                temp = temp.next
            print("None")

    def clear(self):  # Clears the entire list.

        with self.lock:
            self.head = None
            self.size = 0

    def is_empty(self):  # Checks if the list is empty.

        with self.lock:
            return self.size == 0

    def remove_end(self):  # Peek and pop the last element.
        with self.lock:
            if not self.head:
                return None  # List is empty

            if not self.head.next:
                val = self.head.val  # Only one element in the list
                self.head = None
                self.size -= 1
                return val

            temp = self.head
            while temp.next and temp.next.next:
                temp = temp.next

            val = temp.next.val  # Peek the last element
            temp.next = None  # Remove it
            self.size -= 1
            return val
        
    
    def remove_position(self, position):

        if position < 0 or position >= self.size:
            return None  
        
        if position == 0:
            return self.remove_beginning()
            
        current = self.head
        for i in range(position - 1):
            current = current.next
        val = current.next.val
        current.next = current.next.next
        
        self.size -= 1
        
        return val

    def remove_beginning(self):  # Peeks and pops the first element
        with self.lock:
            if not self.head:
                return None  # List is empty
            val = self.head.val  # Peek the first element
            self.head = self.head.next  # Remove it
            self.size -= 1
            return val

    def _make_list(self):
        temp = self.head
        array_list = []

        while temp:
            array_list.append(temp.val)
            temp = temp.next

        return array_list

ll = LinkedList()
ll.add_end(10)
ll.add_end(20)
ll.add_front(5)
ll.print_list()
print(ll.peek())
print(ll.get_size())
print(ll.remove_end())
print(ll.remove_end())
ll.add_end(20)
print(ll.peek())
print(ll.remove_beginning())

## Priority Queue: Intelligent & Prioritized Device Task Handling

### What is a Priority Queue?

A data structure that orders elements based on their priority, ensuring that higher priority elements are processed first.

### Why Use a Priority Queue in a Smart Home?

*   **Real-world Prioritization:**  Different devices have varying levels of importance.
    *   **Examples:**`
        *   Security Alarms (High Priority)
        *   Lighting (Medium Priority)
        *   Decorative Displays (Low Priority)
*   **Resource Allocation:** Efficiently manages system resources by prioritizing critical tasks.

### Priority Calculation

The priority of a device is determined by the following formula:
Priority = Device Type Priority + Device Group Priority + (Location Occupancy * Weight)


**Example:** A security camera in an occupied living room will have a higher priority than a decorative light in an empty bedroom.

In [13]:
import threading

class Task:  # Tasks with a set priority
    def __init__(self, task, priority):
        self.task = task
        self.priority = priority

    def __repr__(self):
        return f"Task(priority={self.priority}, task={self.task})"


class PriorityQueue:
    def __init__(self, task=None):
        self.queue = LinkedList()  # Use Linked List instead of Python List
        self.lock = threading.Lock()

        if task:
            self.enqueue(task) 

    def enqueue(self, newTask: Task):
        with self.lock:
            self._enqueue_internal(newTask)

    def _enqueue_internal(self, newTask: Task):
        """Internal version of enqueue that doesn't acquire the lock"""
        if self.queue.is_empty():
            self.queue.add_end(newTask)
            return

        current = self.queue.head
        prev = None

        while current:
            if newTask.priority < current.val.priority:
                if prev is None:
                    self.queue.add_front(newTask)
                else:
                    new_node = Node(newTask)
                    new_node.next = current
                    prev.next = new_node
                    self.queue.size += 1
                return
            prev = current
            current = current.next

        self.queue.add_end(newTask)

    def dequeue(self):  # Dequeuing the element with the most priority.
        with self.lock:
            if self.queue.is_empty():
                return None 
            return self.queue.remove_beginning()

    def peek(self):  # Returns the element with the most priority
        with self.lock:
            if self.queue.is_empty():
                return None
            return self.queue.peek()

    def print(self):  # Prints the queue with the priorities too.
        with self.lock:
            current = self.queue.head
            while current:
                print(f"Priority: {current.val.priority}, Task: {current.val.task}")
                current = current.next

    def is_empty(self):  # Checks if the queue is empty.
        with self.lock:
            return self.queue.is_empty()

    def size(self):  # Checks the size of the queue.
        with self.lock:
            return self.queue.get_size()

    def clear(self):  # clears the queue.
        with self.lock:
            self.queue.clear()

    def contains(self, task):  # Checks if the task is in the queue.
        with self.lock:
            return self._contains_internal(task)

    def _contains_internal(self, task):
        """Internal version of contains that doesn't acquire the lock"""
        current = self.queue.head
        while current:
            if current.val == task:
                return True
            current = current.next
        return False

    def get_priority(self, task_content):  # Gets the priority of a given task.
        with self.lock:
            return self._get_priority_internal(task_content)

    def _get_priority_internal(self, task_content):
        """Internal version of get_priority that doesn't acquire the lock"""
        current = self.queue.head
        while current:
            if current.val.task == task_content:
                return current.val.priority
            current = current.next
        return -1

    def remove_task(self, task):
        with self.lock:
            task_obj = self._get_task_internal(task)
            if task_obj:
                index = self._get_task_index_internal(task)
                if index != -1:
                    self.queue.remove_position(index)
                    return True
            return False

    def get_task(self, task):  # Get Task Using the task's name
        with self.lock:
            return self._get_task_internal(task)
    
    def _get_task_internal(self, task):
        """Internal version of get_task that doesn't acquire the lock"""
        current = self.queue.head
        while current:
            if current.val.task == task:
                return current.val
            current = current.next
        return None

    def get_task_index(self, task):  # Get Task Using the task's name
        with self.lock:
            return self._get_task_index_internal(task)

    def _get_task_index_internal(self, task):
        """Internal version of get_task_index that doesn't acquire the lock"""
        current = self.queue.head
        index = 0
        while current:
            if current.val.task == task:
                return index
            current = current.next
            index += 1
        return -1

    def update_priority(self, task_name, priority):
        with self.lock:
            index = self._get_task_index_internal(task_name)
            if index != -1:
                self.queue.remove_position(index)
                self._enqueue_internal(Task(task_name, priority))
                return True
            return False

# Test code
if __name__ == "__main__":
    pq = PriorityQueue()
    pq.enqueue(Task("Do laundry", 3))
    pq.enqueue(Task("Finish report", 1))
    pq.enqueue(Task("Buy groceries", 2))

    print("\nQueue after enqueuing:")
    pq.print()

    print("\nDequeuing highest priority task")
    pq.dequeue()
    pq.print()

    print("\nPeeking at the highest priority task:")
    print(pq.peek())

    print(pq.get_priority("Buy groceries"))
    pq.print()

    print("\nUpdating priority of 'Buy groceries' to 0:")
    pq.update_priority("Buy groceries", 0)
    pq.print()
    
    print("\nRemoving 'Do laundry':")
    pq.remove_task("Do laundry")
    print("####################")
    pq.print()

## Energy-Efficient Device Coordination

This system employs a two-pronged approach to optimize energy consumption:

### 1. Rule-Based Energy Optimization (Linked List)

*   Users can define energy-saving rules, such as:
    *   "If the time is after 11 PM and motion is not detected in the living room for 15 minutes, turn off the living room lights."
*   The `LinkedList` ensures that these rules are executed efficiently and in the correct order.

In [14]:
class Rule:
    def __init__(self, device_id= -1, flip_state = False, turn_on = False, turn_off = False, set_power_level =False, power_level = 0, group_name = "",
                turn_group_off = False, turn_group_on= False, type_name= "", turn_type_off= False, turn_type_on= False, location_name= "", turn_location_off= False, turn_location_on= False):
        self.__deviceId = device_id
        self.__flipState = flip_state
        self.__turnOn = turn_on
        self.__turnOff = turn_off
        self.__setPowerLevel = set_power_level
        self.__powerLevel = power_level
        self.__groupName = group_name
        self.__turnGroupOff = turn_group_off
        self.__turnGroupOn = turn_group_on
        self.__typeName = type_name
        self.__turnTypeOff = turn_type_off
        self.__turnTypeOn = turn_type_on
        self.__locationName = location_name
        self.__turnLocationOff = turn_location_off
        self.__turnLocationOn = turn_location_on

    def get_device_id(self):
        return self.__deviceId

    def set_device_id(self, device_id):
        self.__deviceId = device_id

    def get_flip_state(self):
        return self.__flipState

    def set_flip_state(self, flip_state):
        self.__flipState = flip_state

    def get_turn_on(self):
        return self.__turnOn

    def set_turn_on(self, turn_on):
        self.__turnOn = turn_on

    def get_turn_off(self):
        return self.__turnOff

    def set_turn_off(self, turn_off):
        self.__turnOff = turn_off

    def get_set_power_level(self):
        return self.__setPowerLevel

    def set_set_power_level(self, set_power_level):
        self.__setPowerLevel = set_power_level

    def get_power_level(self):
        return self.__powerLevel

    def set_power_level(self, power_level):
        self.__powerLevel = power_level

    def get_group_name(self):
        return self.__groupName

    def set_group_name(self, group_name):
        self.__groupName = group_name

    def get_turn_group_off(self):
        return self.__turnGroupOff

    def set_turn_group_off(self, turn_group_off):
        self.__turnGroupOff = turn_group_off

    def get_turn_group_on(self):
        return self.__turnGroupOn

    def set_turn_group_on(self, turn_group_on):
        self.__turnGroupOn = turn_group_on

    def get_type_name(self):
        return self.__typeName

    def set_type_name(self, type_name):
        self.__typeName = type_name

    def get_turn_type_off(self):
        return self.__turnTypeOff

    def set_turn_type_off(self, turn_type_off):
        self.__turnTypeOff = turn_type_off

    def get_turn_type_on(self):
        return self.__turnTypeOn

    def set_turn_type_on(self, turn_type_on):
        self.__turnTypeOn = turn_type_on

    def get_location_name(self):
        return self.__locationName

    def set_location_name(self, location_name):
        self.__locationName = location_name

    def get_turn_location_off(self):
        return self.__turnLocationOff

    def set_turn_location_off(self, turn_location_off):
        self.__turnLocationOff = turn_location_off

    def get_turn_location_on(self):
        return self.__turnLocationOn

    def set_turn_location_on(self, turn_location_on):
        self.__turnLocationOn = turn_location_on

    def __str__(self):
        return f"Rule(deviceId={self.__deviceId}, flipState={self.__flipState}, turnOn={self.__turnOn}, turnOff={self.__turnOff}, setPowerLevel={self.__setPowerLevel}, powerLevel={self.__powerLevel}, groupName={self.__groupName}, turnGroupOff={self.__turnGroupOff}, turnGroupOn={self.__turnGroupOn}, typeName={self.__typeName}, turnTypeOff={self.__turnTypeOff}, turnTypeOn={self.__turnTypeOn}, locationName={self.__locationName}, turnLocationOff={self.__turnLocationOff}, turnLocationOn={self.__turnLocationOn})"

    def to_dict(self):
        return {
            "device_id": self.__deviceId,
            "flip_state": self.__flipState,
            "turn_on": self.__turnOn,
            "turn_off": self.__turnOff,
            "set_power_level_flag": self.__setPowerLevel,
            "power_level_value": self.__powerLevel,
            "group_name": self.__groupName,
            "turn_group_off": self.__turnGroupOff,
            "turn_group_on": self.__turnGroupOn,
            "type_name": self.__typeName,
            "turn_type_off": self.__turnTypeOff,
            "turn_type_on": self.__turnTypeOn,
            "location_name": self.__locationName,
            "turn_location_off": self.__turnLocationOff,
            "turn_location_on": self.__turnLocationOn
        }
    
#TESTING
rule = Rule("123", True, False, True, False, 0, "Living Room", False, True, "Light", False, True, "Home", False, True)
print(rule)

### 2. Priority Queue Driven Power Management

*   **Monitoring:** The system continuously monitors overall power consumption against a predefined threshold.
*   **Threshold Exceeded:** If the power consumption threshold is exceeded, the `Priority Queue` identifies devices with lower priority.
*   **Dynamic Rule Generation:** The system automatically creates and adds rules to the `LinkedList` to reduce power consumption for lower-priority devices. These rules might involve:
    *   Turning off the device.
    *   Reducing the power level (e.g., dimming lights).
*   **Dynamic Adaptation:** The system continuously adapts to power usage, adjusting device states based on priority and rules to maintain energy efficiency in real-time.


In [15]:
from enum import Enum
import time

class Device:
    def __init__(self, device_id: int, device_name: str, device_type, location, device_group, battery_level, max_battery_capacity,current_battery_capacity, is_on_battery, is_turned_on, base_power_consumption, power_level,turned_on_time, is_interacted):
        self.__device_id = device_id
        self.__device_name = device_name
        self.__device_type = device_type
        self.__location = location
        self.__device_group = device_group
        self.__battery_level = battery_level
        self.__max_battery_capacity = max_battery_capacity
        self.__current_battery_capacity = current_battery_capacity
        self.__is_on_battery = is_on_battery
        self.__is_turned_on = is_turned_on
        self.__base_power_consumption = base_power_consumption
        self.__power_level = power_level
        self.__turned_on_time = turned_on_time
        self.__is_interacted = is_interacted

    def flip_interaction_state(self):
        self.__is_interacted = not self.__is_interacted

    def get_interaction_state(self) -> bool:
        return self.__is_interacted

    def get_minutes_since_turned_on(self):
        if self.__turned_on_time:
            return int((int(time.time()) - self.__turned_on_time) // 60)
        return 0

    def set_turned_on(self, status: bool):
        self.__is_turned_on = status

    def is_turned_on(self) -> bool:
        return self.__is_turned_on

    def set_battery_level(self, level: float):
        self.__battery_level = level

    def get_battery_level(self) -> float:
        return self.__battery_level

    def set_base_power_consumption(self, consumption: float):
        self.__base_power_consumption = consumption

    def get_base_power_consumption(self) -> float:
        return self.__base_power_consumption

    def set_battery_capacity(self, capacity: int):
        self.__max_battery_capacity = capacity

    def get_battery_capacity(self) -> int:
        return self.__max_battery_capacity

    def get_device_id(self) -> int:
        return self.__device_id

    def get_device_name(self) -> str:
        return self.__device_name

    def set_device_name(self, name: str):
        self.__device_name = name

    def get_device_type(self):
        return self.__device_type

    def set_device_type(self, type_: str):
        self.__device_type = type_

    def get_location(self):
        return self.__location

    def set_location(self, location: str):
        self.__location = location

    def get_device_group(self):
        return self.__device_group

    def set_device_group(self, group: str):
        self.__device_group = group

    def get_power_level(self) -> int:
        return self.__power_level

    def set_power_level(self, level: int):
        self.__power_level = level

    def is_on_battery_power(self) -> bool:
        return self.__is_on_battery

    def set_on_battery(self, status: bool):
        self.__is_on_battery = status

    def get_current_battery_capacity(self) -> float:
        return self.__current_battery_capacity

    def set_current_battery_capacity(self, capacity: float):
        self.__current_battery_capacity = capacity

    def set_turned_on_time(self, time: int):
        self.__turned_on_time = time

    def get_turned_on_time(self) -> int:
        return self.__turned_on_time

    def __str__(self):
        return (f"Device ID: {self.get_device_id()}\n"
                f"Device Name: {self.get_device_name()}\n"
                f"Device Type: {self.get_device_type()}\n"
                f"Device Group: {self.get_device_group()}\n"
                f"Location: {self.get_location()}\n"
                f"Power Status: {'On' if self.is_turned_on() else 'Off'}\n"
                f"Battery Level: {self.get_battery_level()}\n"
                f"Power Consumption: {self.get_base_power_consumption()} W\n"
                f"Power Level: {self.get_power_level()}\n")
    def to_dict(self):
    
        def safe_json_val(val):
            if isinstance(val, Enum):
                if isinstance(val, DeviceLocationEnum):
                    return val.value
                else:
                    return val.name 
            elif val is None:
                    return None 
            else:
                    return val

        return {
            'device_id': self.get_device_id(),
            'device_name': self.get_device_name(),
            'device_type': safe_json_val(self.get_device_type()),
            'location': safe_json_val(self.get_location()),
            'device_group': safe_json_val(self.get_device_group()),
            'battery_level': self.get_battery_level(),
            'max_battery_capacity': self.get_battery_capacity(),
            'current_battery_capacity': self.get_current_battery_capacity(),
            'is_on_battery': self.is_on_battery_power(),
            'is_turned_on': self.is_turned_on(),
            'base_power_consumption': self.get_base_power_consumption(),
            'power_level': self.get_power_level(),
            'turned_on_time': self.get_turned_on_time(), 
            'minutes_since_turned_on': self.get_minutes_since_turned_on(),
            'is_interacted': self.get_interaction_state()
        }

#Test

device1 = Device(
    device_id=1,
    device_name="Light",
    device_type="Decorative",
    location="Living Room",
    device_group="LIGHTS",
    battery_level=85.0,
    max_battery_capacity=100,
    current_battery_capacity=85.0,
    is_on_battery=False,
    is_turned_on=True,
    base_power_consumption=10.0,
    power_level=5,
    turned_on_time=int(time.time()),
    is_interacted=False,

)

print(device1)

device1.set_battery_level(90.0)
device1.set_power_level(8)
device1.flip_interaction_state()

print(f"Battery Level after update: {device1.get_battery_level()}")
print(f"Power Level after update: {device1.get_power_level()}")
print(f"Interaction State: {device1.get_interaction_state()}")
print(f"Minutes since turned on: {device1.get_minutes_since_turned_on()} min")


In [16]:
class AirConditioner(Device):
    def __init__(self,
                device_id: int,
                device_name: str,
                device_type,
                location,   
                device_group,
                is_turned_on: bool,
                battery_level: float,       
                base_power_consumption: float,
                max_battery_capacity: int,
                current_battery_capacity: int, 
                power_level: int,
                is_on_battery: bool,           
                turned_on_time: int,           
                is_interacted: bool = False,   
                mode: bool = True             
            ):

        super().__init__(
            device_id=device_id,
            device_name=device_name,
            device_type=device_type,
            location=location,            
            device_group=device_group,      
            is_turned_on=is_turned_on,
            battery_level=battery_level,
            base_power_consumption=base_power_consumption, 
            max_battery_capacity=max_battery_capacity,
            current_battery_capacity=current_battery_capacity, 
            power_level=power_level,
            is_on_battery=is_on_battery,         
            turned_on_time=turned_on_time,        
            is_interacted=is_interacted          
        )

        self._mode = mode 
        self.simulation_temp_change_time = time.time()

    def get_mode(self) -> bool:
        return self._mode

    def set_mode(self, mode: bool):
        self._mode = mode

    def toggle_mode(self):
        self._mode = not self._mode

    def get_simulation_temp_change_time(self) -> int:
        return self.simulation_temp_change_time

    def set_simulation_temp_change_time(self, simulation_temp_change_time: int):
        self.simulation_temp_change_time = simulation_temp_change_time

    def get_minutes_since_temp_change(self) -> int:
        return int((int(time.time()) - self.simulation_temp_change_time) // 60)

    def __str__(self) -> str:
        return super().__str__() + " Mode: " + ("Cooling" if self.get_mode() else "Heating")


### 3. Enum Based Priority Calculation

*   **Easy Modification:** Modifying priority values is as simple as changing one number
*   **Fast and Easy Calculation:** All priority values can be accessed and calculated very easily
*   **Ease of Use** User does not have to manually include priority values for every single device added

In [17]:
from enum import Enum


class DeviceGroupEnum(Enum):
    LIGHTS = 10
    FANS = 9
    ALARMS = 15
    CAMERAS = 14
    AIRCONDITIONERS = 8
    HEATERS = 8
    APPLIANCES = 6
    GARDENING = 3
    ENTERTAINMENT = 2
    CLEANING = 5
    LAUNDRY = 4
    WEARABLES = 7
    BATHROOM = 12
    OTHERS = 1

    def get_priority(self):
        return self.value


class DeviceGroup:
    def __init__(self, group_name: str):
        self.group_name = group_name
        self.devices = []

    def add_device(self, device: Device):
        self.devices.append(device)

    def remove_device(self, device: Device):
        if device in self.devices:
            self.devices.remove(device)

    def get_devices(self):
        return self.devices

    def turn_off_all_devices(self):
        for device in self.devices:
            device.set_turned_on(False)

    def turn_on_all_devices(self):
        for device in self.devices:
            device.set_turned_on(True)

    def get_device_by_name(self, name: str):
        name_lower = name.lower()
        return next((device for device in self.devices if name_lower in device.get_device_name().lower()), None)

    def get_device_by_id(self, device_id: int):
        return next((device for device in self.devices if device.get_device_id() == device_id), None)

    def get_group_name(self):
        return self.group_name

In [18]:
from enum import Enum


class DeviceTypeEnum(Enum):
    DECORATIVE = 1,
    HEALTH = 15,
    ENTERTAINMENT = 3,
    SECURITY = 20,
    PERSONALCARE = 7,
    CONNECTIVITY = 10,
    COOKING = 12,
    LUXURY = 2,
    OFFICE = 10,
    OTHERS = 5

    def get_priority(self):
        return self.value


class DeviceType:
    def __init__(self, typeName: str):
        self.typeName = typeName
        self.devices = []

    def add_device(self, device: Device):
        self.devices.append(device)

    def remove_device(self, device: Device):
        if device in self.devices:
            self.devices.remove(device)

    def get_devices(self):
        return self.devices

    def turn_off_all_devices(self):
        for device in self.devices:
            device.set_turned_on(False)

    def turn_on_all_devices(self):
        for device in self.devices:
            device.set_turned_on(True)

    def get_device_by_name(self, name: str):
        name_lower = name.lower()
        return next((device for device in self.devices if name_lower in device.get_device_name().lower()), None)

    def get_device_by_id(self, device_id: int):
        return next((device for device in self.devices if device.get_device_id() == device_id), None)



In [19]:
from enum import Enum


class DeviceLocationEnum(Enum):
    LIVINGROOM = "Living Room"
    BEDROOM = "Bedroom"
    BEDROOM2 = "Bedroom 2"
    BEDROOM3 = "Bedroom 3"
    BEDROOM4 = "Bedroom 4"
    GARDEN = "Garden"
    OFFICE = "Office"
    ENTRANCE = "Entrance"
    KITCHEN = "Kitchen"
    BATHROOM = "Bathroom"
    BATHROOM2 = "Bathroom 2"
    BATHROOM3 = "Bathroom 3"
    OTHERS = "Others"


class DeviceLocation:
    def __init__(self, location: str):
        self.location = location
        self.devices = []
        self.people = 0
        self.temperature = 0.0

    def add_device(self, device: Device):

        self.devices.append(device)

    def remove_device(self, device: Device):

        if device in self.devices:
            self.devices.remove(device)

    def get_devices(self):
        return self.devices

    def get_people(self):
        return self.people

    def set_people(self, people: int):
        self.people = people

    def add_people(self, people: int):
        self.people += people

    def remove_people(self, people: int):
        self.people = max(0, self.people - people)

    def turn_off_all_devices(self):

        for device in self.devices:
            device.set_turned_on(False)

    def turn_on_all_devices(self):

        for device in self.devices:
            device.set_turned_on(True)

    def get_temperature(self):
        return self.temperature

    def set_temperature(self, temperature: float):
        self.temperature = temperature

    def get_device_by_name(self, name: str):

        name_lower = name.lower()
        return next((device for device in self.devices if name_lower in device.get_device_name().lower()), None)

    def get_device_by_id(self, device_id: int):

        return next((device for device in self.devices if device.get_device_id() == device_id), None)

    def __str__(self):
        return f"Location: {self.location}"

### 4. Logging system

*   **Log Monitoring:** Displays system logs for transparency and debugging, including:
    *   Info logs
    *   Warning logs
    *   Severe logs
    *   Power logs
    *   Battery logs

In [20]:
import logging


class LogTask:
    LEVEL_LIST = [
        logging.ERROR,
        logging.CRITICAL,
        logging.WARNING,
        logging.INFO
    ]

    def __init__(self, log_level, message):
        self.logLevel = log_level
        self.message = message

    def get_log_level(self):
        return self.logLevel

    def get_message(self):
        return self.message

    def set_log_level(self, log_level):
        self.logLevel = log_level

    def set_message(self, message):
        self.message = message
    
    def to_dict(self):
        return {
            "level": logging.getLevelName(self.logLevel),
            "message": self.message
        }


# Exceptions
 

In [21]:
class RuleParsingException(Exception):

    def __init__(self, message: str = "Rule parsing error", cause: Exception | None = None):
        super().__init__(message)
        self.cause = cause

    def __str__(self) -> str:
        return f"RuleParsingException: {super().__str__()}"

### SmartHome.py Overview

**Integrating code**
- Acts as the primary controller, integrating all device and data structure functionalities  
- Manages device objects for efficient monitoring and control  
- Enforces power consumption thresholds to optimize energy usage  
- Maintains, checks, and updates battery levels across connected devices  

**Logging**
- Logs key events, enhancing transparency and debugging  

**Priority Management**
- Coordinates with the Priority Queue to handle high-priority device tasks  
- Leverages the Linked List to manage and execute automation rules in a defined order  
- Ensures continuous, coordinated operation among devices, rules, and system events

## User Interface

The user interface provides a user-friendly way to interact with the smart home system:

*   **User-Friendly Access:**  A visual interface for easy interaction.
*   **Device Status Display:** Shows the real-time status of devices (on/off, power level, battery level).
*   **Device Control:** Allows users to manually control devices (e.g., toggle on/off).
*   **Log Monitoring:**  Allows users to monitor all logs

In [23]:
from flask import Flask, request, jsonify
from flask_cors import CORS, cross_origin
import json

app = Flask("Smart_Home_Automation")
CORS(app, supports_credentials=False, origins="*")
app.config['Headers'] = 'Access-Control-Allow-Origin'
app.config['CORS_HEADERS'] = 'Content-Type'
app.config['CONTENT_TYPE'] = 'multipart/form-data'

smart_home = SmartHome(10,25, False)
smart_home.add_device(smart_home.create_device("Lamp", DeviceTypeEnum.DECORATIVE, DeviceGroupEnum.LIGHTS, DeviceLocationEnum.LIVINGROOM, True, 100, 1, 1000, 1))
smart_home.add_device(smart_home.create_device("Fan", DeviceTypeEnum.DECORATIVE, DeviceGroupEnum.LIGHTS, DeviceLocationEnum.LIVINGROOM, True, 100, 2, 1000, 1))
smart_home.add_device(smart_home.create_device("AC", DeviceTypeEnum.DECORATIVE, DeviceGroupEnum.AIRCONDITIONERS, DeviceLocationEnum.LIVINGROOM, True, 100, 3, 1000, 1))

@app.route("/devices", methods=["GET"])
@cross_origin()
def get_all_devices():
    devices = smart_home.get_devices()
    device_list = [device.to_dict() for device in devices if isinstance(device, Device)]
    return jsonify(device_list)

@app.route("/devices/off", methods=["GET"])
@cross_origin()
def get_off_devices():
    devices = smart_home.get_powered_off_devices()
    device_list = [device.to_dict() for device in devices if isinstance(device, Device)]
    return jsonify(device_list)

@app.route("/devices/on", methods=["GET"])
@cross_origin()
def get_on_devices():
    devices = smart_home.get_powered_on_devices()
    device_list = [device.to_dict() for device in devices if isinstance(device, Device)]
    return jsonify(device_list)

@app.route("/devices/id/<id>", methods=["GET"])
@cross_origin()
def get_device_by_id(id):
    try:
        device_id = int(id)
        device = smart_home.get_device_by_id(device_id)
        if device and isinstance(device, Device):
            return jsonify(device.to_dict())
        else:
            status = 404 if not device else 500 
            msg = f"Device with id {device_id} not found" if not device else "Internal error: Invalid device object"
            return jsonify({"error": msg}), status
    except ValueError:
        return jsonify({"error": f"Invalid ID format: {id}"}), 400
    except Exception as e:
        print(f"Error in get_device_by_id: {e}")
        return jsonify({"error": "Internal server error"}), 500

@app.route("/devices/name/<name>", methods=["GET"])
@cross_origin()
def get_device_by_name(name):

    device = smart_home.get_device_by_name(str(name))
    if device and isinstance(device, Device):
        return jsonify(device.to_dict())
    else:
        status = 404 if not device else 500
        msg = f"Device with name '{name}' not found" if not device else "Internal error: Invalid device object"
        return jsonify({"error": msg}), status

@app.route("/devices/power_consumption", methods=["GET"])
@cross_origin()
def get_power_consumption():
    return jsonify(smart_home.get_power_consumption())

@app.route("/devices/threshold", methods=["GET"])
@cross_origin()
def get_power_consumption_threshold():
    return jsonify(smart_home.get_threshold())

@app.route("/devices/ideal_temp", methods=["GET"])
@cross_origin()
def get_ideal_temperature():
    return jsonify(smart_home.get_ideal_temp())

@app.route("/devices/groups", methods=["GET"])
@cross_origin()
def get_device_groups():
    groups = smart_home.get_device_groups()
    return jsonify(list(groups.keys()) if isinstance(groups, dict) else groups)

@app.route("/devices/locations", methods=["GET"])
@cross_origin()
def get_device_locations():
    locations = smart_home.get_device_locations()
    return jsonify(list(locations.keys()) if isinstance(locations, dict) else locations)

@app.route("/devices/types", methods=["GET"])
@cross_origin()
def get_device_types():
    types = smart_home.get_device_types()
    return jsonify(list(types.keys()) if isinstance(types, dict) else types)

@app.route("/devices/groups/devices", methods=["GET"])
@cross_origin()
def get_devices_by_group():
    group = request.args.get("group")
    if not group:
         return jsonify({"error": "Missing 'group' query parameter"}), 400
    devices = smart_home.get_devices_by_group(group)
    device_list = [device.to_dict() for device in devices]
    return jsonify(device_list)

@app.route("/devices/locations/devices", methods=["GET"])
@cross_origin()
def get_devices_by_location():
    location = request.args.get("location")
    if not location:
        return jsonify({"error": "Missing 'location' query parameter"}), 400
    
    devices = smart_home.get_devices_by_location(location)
    device_list = [device.to_dict() for device in devices]
    return jsonify(device_list)

@app.route("/devices/types/devices", methods=["GET"])
@cross_origin()
def get_devices_by_type():
    device_type = request.args.get("type")
    if not device_type:
        return jsonify({"error": "Missing 'type' query parameter"}), 400
    devices = smart_home.get_devices_by_type(device_type)
    device_list = [device.to_dict() for device in devices]
    return jsonify(device_list)

@app.route("/devices/name/<name>/power_level", methods=["GET"])
@cross_origin()
def get_power_level_by_name(name):
    device = smart_home.get_device_by_name(name)
    if device:
        return jsonify(device.get_power_level())
    else:
        return jsonify({"error": f"Device with name '{name}' not found"}), 404

@app.route("/devices/id/<id>/power_level", methods=["GET"])
@cross_origin()
def get_power_level_by_id(id):
    try:
        device_id = int(id)
        device = smart_home.get_device_by_id(device_id)
        if device:
            return jsonify(device.get_power_level())
        else:
            return jsonify({"error": f"Device with id {device_id} not found"}), 404
    except ValueError:
        return jsonify({"error": f"Invalid ID format: {id}"}), 400

@app.route("/devices/name/<name>/battery_level", methods=["GET"])
@cross_origin()
def get_battery_level_by_name(name):
    device = smart_home.get_device_by_name(name)
    if device:
        return jsonify(device.get_battery_level())
    else:
        return jsonify({"error": f"Device with name '{name}' not found"}), 404

@app.route("/devices/id/<id>/battery_level", methods=["GET"])
@cross_origin()
def get_battery_level_by_id(id):
    try:
        device_id = int(id)
        device = smart_home.get_device_by_id(device_id)
        if device:
            return jsonify(device.get_battery_level())
        else:
            return jsonify({"error": f"Device with id {device_id} not found"}), 404
    except ValueError:
        return jsonify({"error": f"Invalid ID format: {id}"}), 400

@app.route("/devices/name/<name>/max_battery_capacity", methods=["GET"])
@cross_origin()
def get_max_battery_capacity_by_name(name):
    device = smart_home.get_device_by_name(name)
    if device:

        return jsonify(device.get_battery_capacity())
    else:
        return jsonify({"error": f"Device with name '{name}' not found"}), 404

@app.route("/devices/id/<id>/max_battery_capacity", methods=["GET"])
@cross_origin()
def get_max_battery_capacity_by_id(id):
    try:
        device_id = int(id)
        device = smart_home.get_device_by_id(device_id)
        if device:
            return jsonify(device.get_battery_capacity())
        else:
            return jsonify({"error": f"Device with id {device_id} not found"}), 404
    except ValueError:
        return jsonify({"error": f"Invalid ID format: {id}"}), 400

@app.route("/devices/name/<name>/power_consumption", methods=["GET"])
@cross_origin()
def get_power_consumption_by_name(name):
    device = smart_home.get_device_by_name(name)
    if device:

        return jsonify(device.get_base_power_consumption())
    else:
        return jsonify({"error": f"Device with name '{name}' not found"}), 404

@app.route("/devices/id/<id>/power_consumption", methods=["GET"])
@cross_origin()
def get_power_consumption_by_id(id):
    try:
        device_id = int(id)
        device = smart_home.get_device_by_id(device_id)
        if device:
            return jsonify(device.get_base_power_consumption())
        else:
            return jsonify({"error": f"Device with id {device_id} not found"}), 404
    except ValueError:
        return jsonify({"error": f"Invalid ID format: {id}"}), 400

@app.route("/devices/log/info", methods=["GET"])
@cross_origin()
def get_info_logs():
    logs = list(smart_home.get_info_tasks()) 
    smart_home.clear_info_tasks()

    return jsonify(convert_log_task_list(logs))

@app.route("/devices/log/warning", methods=["GET"])
@cross_origin()
def get_warning_logs():
    logs = list(smart_home.get_warning_tasks())
    smart_home.clear_warning_tasks()
    return jsonify(convert_log_task_list(logs))

@app.route("/devices/log/severe", methods=["GET"])
@cross_origin()
def get_error_logs():
    logs = list(smart_home.get_severe_tasks())
    smart_home.clear_severe_tasks()
    return jsonify(convert_log_task_list(logs))

@app.route("/devices/log/power_consumption", methods=["GET"])
@cross_origin()
def get_power_consumption_logs():
    logs = list(smart_home.get_power_consumption_tasks()) 
    smart_home.clear_power_consumption_tasks()
    return jsonify(convert_log_task_list(logs)) 

@app.route("/devices/log/battery", methods=["GET"])
@cross_origin()
def get_battery_logs():
    logs = list(smart_home.get_device_battery_tasks()) 
    smart_home.clear_device_battery_tasks()
    return jsonify(convert_log_task_list(logs))

@app.route("/devices/debug/linkedlists", methods=["GET"])
@cross_origin()
def get_linked_lists():

    try:
        logging_list_data = convert_log_task_list(smart_home.get_logging_list())
        power_log_data = convert_log_task_list(smart_home.get_power_consumption_log_list())
        battery_log_data = convert_log_task_list(smart_home.get_device_battery_log_list())
        rule_list_data = convert_rule_list(smart_home.get_rule_list())

        linked_lists_data = {
            "loggingList": logging_list_data,
            "powerConsumptionLogList": power_log_data,
            "deviceBatteryLogList": battery_log_data,
            "ruleList": rule_list_data
        }
        return jsonify(linked_lists_data)
    except Exception as e:
        print(f"Error in /debug/linkedlists: {e}")

        return jsonify({"error": "Failed to serialize linked list data"}), 500

@app.route("/devices/debug/priorityqueues", methods=["GET"])
@cross_origin()
def get_priority_queues():

    try:
        priority_queues_data = {
            "deviceQueue": convert_device_queue(smart_home.get_device_queue()),
            "powerReducibleDevices": convert_device_queue(smart_home.get_power_reducible_devices()),
            "turnBackOnDevices": convert_device_queue(smart_home.get_turn_back_on_devices())
        }
        return jsonify(priority_queues_data)
    except Exception as e:
        print(f"Error in /debug/priorityqueues: {e}")

        return jsonify({"error": "Failed to serialize priority queue data"}), 500

def convert_log_task_list(log_source):
    """
    Converts a list, tuple, or LinkedList of LogTask objects
    (or strings) into a list of dictionaries.
    """
    converted_list = []
    items_to_process = []

    if isinstance(log_source, (list, tuple)):
        items_to_process = log_source
    elif isinstance(log_source, LinkedList):

        with log_source.lock:
            items_to_process = log_source._make_list()
    else:
        print(f"Warning: convert_log_task_list received incompatible type: {type(log_source)}")
        return []

    for item in items_to_process:
        if isinstance(item, LogTask):

            if hasattr(item, 'to_dict') and callable(item.to_dict):
                converted_list.append(item.to_dict())
            else:
                print(f"Warning: LogTask object lacks to_dict method: {item}")
                converted_list.append({"error": "LogTask missing to_dict"})

        elif isinstance(item, str):
            converted_list.append(item) 
        else:

            print(f"Warning: Item in log source is not a LogTask or str: {type(item)}")
            converted_list.append({"error": f"Invalid log item type: {type(item)}"})

    return converted_list

def convert_rule_list(rule_source):
    """
    Converts a list, tuple, or LinkedList of Rule objects
    into a list of dictionaries.
    """
    converted_list = []
    items_to_process = []

    if isinstance(rule_source, (list, tuple)):
        items_to_process = rule_source
    elif isinstance(rule_source, LinkedList):

        with rule_source.lock:
             items_to_process = rule_source._make_list()
    else:
        print(f"Warning: convert_rule_list received incompatible type: {type(rule_source)}")
        return []

    for item in items_to_process:
        if isinstance(item, Rule):

            if hasattr(item, 'to_dict') and callable(item.to_dict):
                converted_list.append(item.to_dict())
            else:
                 print(f"Warning: Rule object lacks to_dict method: {item}")
                 converted_list.append({"error": "Rule missing to_dict"})
        else:

            print(f"Warning: Item in rule source is not a Rule object: {type(item)}")

            converted_list.append({"error": f"Invalid rule item type: {type(item)}"})

    return converted_list

def convert_device_queue(queue_obj):
    """
    Converts a PriorityQueue containing Task objects into a list of
    dictionaries, where each dict represents a Task's content and priority.
    Assumes Task objects have 'task' and 'priority' attributes.
    The 'task' attribute might be a Device object, str, int, etc.
    """
    converted_list = []

    if not isinstance(queue_obj, PriorityQueue):
        print(f"Warning: convert_device_queue received non-PriorityQueue object: {type(queue_obj)}")

        return []

    try:
        items_in_queue = queue_obj.queue._make_list()

        for item in items_in_queue:

            if isinstance(item, Task):
                task_content = item.task 
                priority = item.priority 

                name = "Unknown"
                if isinstance(task_content, Device):

                    if hasattr(task_content, 'get_device_name') and callable(task_content.get_device_name):
                         name = task_content.get_device_name()
                    else:
                         print(f"Warning: Device object lacks get_device_name method: {task_content}")
                         name = f"Device (no name method: {type(task_content)})"
                elif isinstance(task_content, (str, int)):
                    name = str(task_content) 
                else:

                    name = f"Object ({type(task_content)})"

                map_obj = {
                    "Name": name,       
                    "Priority": priority
                }
                converted_list.append(map_obj)
            else:

                print(f"Warning: Item in device queue is not a Task object: {type(item)}")
                converted_list.append({"error": f"Invalid queue item type: {type(item)}"})

    except Exception as e:
        print(f"Error during convert_device_queue: {e}")

    return converted_list


@app.route("/devices", methods=["POST"])
@cross_origin()
def add_device():
    if not request.is_json:
        return jsonify({"error": "Request must be JSON"}), 400

    device_data = request.json

    try:

        device_type = DeviceTypeEnum[device_data['device_type']]
        device_group = DeviceGroupEnum[device_data['device_group']]
        location = DeviceLocationEnum[device_data['location']]

        device = smart_home.create_device(
            device_data['device_name'],
            device_type,
            device_group,
            location,
            device_data.get('is_turned_on', False),
            device_data.get('battery_level', 100.0),
            device_data.get('base_power_consumption', 0.0),
            device_data.get('max_battery_capacity', 0),
            device_data.get('power_level', 0),
        )

        smart_home.add_device(device)

        return jsonify({
            "message": "Device added successfully",
            "device": device.to_dict()
        }), 201

    except (KeyError, ValueError) as e:
        return jsonify({"error": f"Invalid device data: {str(e)}"}), 400

@app.route("/devices/id/<id>/on", methods=["PUT"])
@cross_origin()
def turn_on_device_by_id(id):
    try:
        device_id = int(id)
        device = smart_home.get_device_by_id(device_id)
        if device:
            smart_home.turn_on_device(device)
            return jsonify({"message": f"Device '{device.get_device_name()}' turned on successfully"})
        else:
            return jsonify({"error": f"Device with id {device_id} not found"}), 404
    except ValueError:
        return jsonify({"error": f"Invalid ID format: {id}"}), 400

@app.route("/devices/name/<name>/on", methods=["PUT"])
@cross_origin()
def turn_on_device_by_name(name):
    device = smart_home.get_device_by_name(name)
    if device:
        smart_home.turn_on_device(device)
        return jsonify({"message": f"Device '{name}' turned on successfully"})
    else:
        return jsonify({"error": f"Device with name '{name}' not found"}), 404

@app.route("/devices/id/<id>/off", methods=["PUT"])
@cross_origin()
def turn_off_device_by_id(id):
    try:
        device_id = int(id)
        device = smart_home.get_device_by_id(device_id)
        if device:
            smart_home.turn_off_device(device)
            return jsonify({"message": f"Device '{device.get_device_name()}' turned off successfully"})
        else:
            return jsonify({"error": f"Device with id {device_id} not found"}), 404
    except ValueError:
        return jsonify({"error": f"Invalid ID format: {id}"}), 400

@app.route("/devices/name/<name>/off", methods=["PUT"])
@cross_origin()
def turn_off_device_by_name(name):
    device = smart_home.get_device_by_name(name)
    if device:
        smart_home.turn_off_device(device)
        return jsonify({"message": f"Device '{name}' turned off successfully"})
    else:
        return jsonify({"error": f"Device with name '{name}' not found"}), 404

@app.route("/devices/threshold/<threshold>", methods=["PUT"])
@cross_origin()
def set_power_consumption_threshold(threshold):
    try:
        threshold_value = float(threshold)
        smart_home.set_threshold(threshold_value)
        return jsonify({"message": f"Threshold set successfully to {threshold_value}"})
    except ValueError:
        return jsonify({"error": f"Invalid threshold format: {threshold}. Must be a number."}), 400

@app.route("/devices/ideal_temp/<ideal_temp>", methods=["PUT"])
@cross_origin()
def set_ideal_temperature(ideal_temp):
    try:
        temp_value = int(ideal_temp)
        smart_home.set_ideal_temp(temp_value)
        return jsonify({"message": f"Ideal temperature set successfully to {temp_value}"})
    except ValueError:
         return jsonify({"error": f"Invalid ideal temperature format: {ideal_temp}. Must be an integer."}), 400

@app.route("/devices/id/<id>/power_level/<power_level>", methods=["PUT"])
@cross_origin()
def set_power_level_by_id(id, power_level):
    rule_string = f"set {id} {power_level}"
    rule = smart_home.parse_rule(rule_string)
    if rule:

         device_exists = smart_home.get_device_by_id(rule.get('device_id'))
         if device_exists:
              smart_home.add_rule(rule) 
              return jsonify({"message": f"Power level rule added/executed for device ID {id}"})
         else:
              return jsonify({"error": f"Device with ID {id} not found"}), 404
    else:
         return jsonify({"error": f"Failed to parse rule or invalid power level: {rule_string}"}), 400

@app.route("/devices/name/<name>/power_level/<power_level>", methods=["PUT"])
@cross_origin()
def set_power_level_by_name(name, power_level):
    rule_string = f"set {name} {power_level}"
    rule = smart_home.parse_rule(rule_string)
    if rule:

         device_exists = smart_home.get_device_by_name(rule.get('device_name'))
         if device_exists:
              smart_home.add_rule(rule) 
              return jsonify({"message": f"Power level rule added/executed for device '{name}'"})
         else:
              return jsonify({"error": f"Device with name '{name}' not found"}), 404
    else:
        return jsonify({"error": f"Failed to parse rule or invalid power level: {rule_string}"}), 400

@app.route("/devices/location/<location>/add_person", methods=["PUT"])
@cross_origin()
def add_person_to_location(location):

    loc_obj = smart_home.get_location(location)
    if loc_obj:
        smart_home.add_person(loc_obj.location)
        return jsonify({"message": f"Person added successfully to location '{location}'"})
    else:
        return jsonify({"error": f"Location '{location}' not found or invalid"}), 404

@app.route("/devices/location/<location>/remove_person", methods=["PUT"])
@cross_origin()
def remove_person_from_location(location):
    loc_obj = smart_home.get_location(location)
    if loc_obj:
        smart_home.remove_person(loc_obj.location)
        return jsonify({"message": f"Person removed successfully from location '{location}'"})
    else:
        return jsonify({"error": f"Location '{location}' not found or invalid"}), 404

@app.route("/devices/name/<name>", methods=["DELETE"])
@cross_origin()
def remove_device_by_name(name):
    device = smart_home.get_device_by_name(name)
    if device:
        if smart_home.remove_device(device):
            return jsonify({"message": f"Device '{name}' removed successfully"})
        else:

            return jsonify({"error": f"Failed to remove device '{name}'"}), 500
    else:
        return jsonify({"error": f"Device with name '{name}' not found"}), 404

@app.route("/devices/id/<id>", methods=["DELETE"])
@cross_origin()
def remove_device_by_id(id):
    try:
        device_id = int(id)
        device = smart_home.get_device_by_id(device_id)
        if device:
            if smart_home.remove_device(device):
                return jsonify({"message": f"Device with ID {device_id} removed successfully"})
            else:
                return jsonify({"error": f"Failed to remove device with ID {device_id}"}), 500
        else:
            return jsonify({"error": f"Device with id {device_id} not found"}), 404
    except ValueError:
        return jsonify({"error": f"Invalid ID format: {id}"}), 400

app.run(host='127.0.0.1', port=8080)